In [1]:
import itertools
import os.path as op
import os
import glob # imported for csv
import gc
import csv
import pandas as pd

from gradec.decode import LDADecoder
from gradec.utils import _rm_medial_wall, _decoding_filter
from gradec.plot import plot_surf_maps, plot_radar, plot_cloud
from gradec.fetcher import _fetch_features, _fetch_frequencies, _fetch_classification
import nibabel as nib
import numpy as np

### Create csv file with all tract names

In [2]:
file_dir = os.getcwd() + "/data/white-matter-atlas_thresholds/cortexmap_binarize_smooth-surf-1_threshold-0.15_dilate-0/cortexmap/func"
file_names = []
for x in os.listdir(file_dir):
    if x.endswith(".gii"):
        file_names.append(x)

regions_bi = []
regions_mono = []

for i in file_names:
    split_region = i.split("_")[0][3:]
    if split_region.find('left') == 0:
        regions_bi.append(split_region[4:])
    elif split_region.find('right') == 0:
        regions_bi.append(split_region[5:])
    else:
        regions_mono.append(split_region)

#np.array(regions_mono).tofile("regions_mono.csv", sep = ",")
#np.array(regions_bi).tofile("regions_bi.csv", sep = ",")

sub = sorted(list(set(regions_bi)))
len(sub)

28

### Define space, density and paths to data

In [3]:
SPACE, DENSITY = "fsaverage", "164k"
DSET, MODEL = "neuroquery", "lda"

data_dir = op.join(".", "data")
neuromaps_dir = op.join(data_dir, "neuromaps")
figures_dir = op.join(data_dir, "figures")

# List of possible combinations of tracts, regions and smoothing
#tracts = ["Arc", "SLF1And2", "CST"]
tracts = sub
regions = ["RAS", "LPI"]
smths = ["", ".smooth_1"]
thresholds = ["0", "0.15", "0.25"]

# Dictionaries for the title of the figures
""" TRACTS_DICT = {
    "Arc": "Arcuate",
    "CST": "Corticospinal",
    "SLF1And2": "SLF 1 & 2",
    "forcepsMajor": "Forceps Major",
} """
TRACTS_DICT = dict(zip(regions_bi, regions_bi))
REGIONS_DICT = {
    "LPI": "Left-Posterior-Inferior",
    "RAS": "Right-Anterior-Superior",
}
SMTHS_DICT = {
    "": "Unsmoothed",
    ".smooth_1": "Smoothed",
}

### Train and LDA-based decoder on NeuroQuery detabase

In [4]:
decode = LDADecoder(space=SPACE, density=DENSITY, calc_pvals=False, data_dir=data_dir)
decode.fit(DSET)

# Load features for visualization
features = _fetch_features(DSET, MODEL, data_dir=data_dir)
frequencies = _fetch_frequencies(DSET, MODEL, data_dir=data_dir)
classification, class_lst = _fetch_classification(DSET, MODEL, data_dir=data_dir)

### @Daniela HERE's YOUR RESOURCE: Run decoder on each regions separate to output correlation

In [5]:
# IF only wanting to a few figures
tracts = ["ILF"]
regions = ["LPI"]
smths = [".smooth_1"]
thresholds = ["0"]

In [6]:
# RADAR PLOTS AND CORRELATION VALUESs
sep_figures_dir_test = op.join(figures_dir, "separated_test")
os.makedirs(sep_figures_dir_test, exist_ok=True)

separated_results = {}
corrs_all = []
features_all = []
fileName = []
for fig_i, (threshold, tract, region, smth) in enumerate(itertools.product(thresholds, tracts, regions, smths)):
    # Path to the maps
    regions_dir = op.join(
        data_dir, 
        "white-matter-atlas_thresholds", 
        f"cortexmap_binarize_smooth-surf-1_threshold-{threshold}_dilate-0", 
        "cortexmap", 
        "func",
    )
    
    # Read maps
    map_lh = op.join(regions_dir, f"lh.left{tract}_box_1mm_{region}_FiberEndpoint{smth}.func.gii")
    map_rh = op.join(regions_dir, f"rh.right{tract}_box_1mm_{region}_FiberEndpoint{smth}.func.gii")
    map_arr_lh = nib.load(map_lh).agg_data()
    map_arr_rh = nib.load(map_rh).agg_data()

    # Remove medial wall
    map_arr = _rm_medial_wall(
        map_arr_lh,
        map_arr_rh,
        space=SPACE,
        density=DENSITY,
        neuromaps_dir=neuromaps_dir,
    )

    # Decode map
    corrs_df = decode.transform([map_arr], method="correlation")
    filtered_df, filtered_features, filtered_frequencies = _decoding_filter(
        corrs_df,
        features,
        classification,
        freq_by_topic=frequencies,
        class_by_topic=class_lst,
    )
    filtered_df.columns = ["r"]
    separated_results[f"{tract}_{region}{smth}_thr-{threshold}"] = filtered_df.sort_values(by="r", ascending=False)

    # radar plot and corr values 
    corrs = filtered_df["r"].to_numpy()
    n_rows = min(len(corrs), 10)
    print(n_rows)
    if not np.any(np.isnan(corrs)) and corrs.size > 0: # Skip one of the regions of CST
        # Radar plot
        # plot_radar(
        #     corrs, 
        #     filtered_features, 
        #     MODEL,
        #     out_fig=op.join(sep_figures_dir_test, f"{fig_i}-02_{tract}_{region}{smth}_thr-{threshold}_radar.png"),
        # )

        # Sort features and correlations
        corrs_plot = np.array(corrs)
        sorted_indices = np.argsort(-corrs_plot)
        corrs_plot = corrs_plot[sorted_indices]
        feature_plot = np.array(filtered_features, dtype=object)[sorted_indices]

        # Create df for a few tracts
        radar_df = pd.DataFrame() 
        radar_df['r'] = corrs_plot
        radar_df['terms'] = feature_plot
        #radar_df.to_csv(tract + "_" + region + "_" + smth + "_" + threshold + "_radar_plot_data.csv")

        # # Keep only topics and terms in radar plot
        # n_top_terms = 3
        # corrs_plot = corrs_plot[:n_rows]
        # feature_plot = feature_plot[:n_rows]
        # feature_plot = ["--".join(feature_plot[:n_top_terms]).replace(" ", "--") for feature_plot in feature_plot]
        # corrs_all = np.append(corrs_all, corrs_plot)
        # features_all = np.append(features_all, feature_plot)
        #fileName = np.append(fileName, np.full(n_rows, tract + "_" + region + "_" + smth))

    # # Create df for all figures
    # radar_df = pd.DataFrame() 
    # radar_df['file name'] = fileName
    # radar_df['r'] = corrs_all
    # radar_df['terms'] = features_all
    # radar_df.to_csv('ILF_RAS_smooth_thr0_radar_plot_data.csv')

  0%|          | 0/1 [00:00<?, ?it/s]


10


In [11]:
separated_results

{'ILF_LPI.smooth_1_thr-0':                                                r
 feature                                         
 72_visual_process_fusiform gyrus        0.618054
 183_visual_fixation_orientation         0.504462
 5_motion_sts_psts                       0.470403
 114_face_ffa_process                    0.437117
 157_object_loc_image                    0.403368
 192_stimuli_stimulus_process            0.373574
 113_gaze_saccade_eye                    0.366811
 85_reading_letter_orthographic          0.279264
 59_categories_category_animal           0.276348
 187_spatial_shape_visuospatial          0.271425
 198_cue_cues_cued                       0.270381
 169_scene_ppa_image                     0.248332
 181_attention_attended_top              0.241749
 37_hand_body_part                       0.234183
 168_video_gesture_communication         0.225200
 121_perceptual_perception_conscious     0.218379
 2_picture_valence_arousal               0.211130
 167_virtual_navigation_

In [18]:
# NOTES TO SELF
sep_figures_dir = op.join(figures_dir, "separated")
os.makedirs(sep_figures_dir, exist_ok=True)

separated_results = {}
fig_names = []
tract = "Arc"
regions = "RAS"
smths = ".smooth_1"
thresholds = "0"

regions_dir = op.join(
        data_dir, 
        "white-matter-atlas_thresholds", 
        f"cortexmap_binarize_smooth-surf-1_threshold-{thresholds}_dilate-0", 
        "cortexmap", 
        "func",
)

# Read maps
map_lh = op.join(regions_dir, f"lh.left{tract}_box_1mm_{regions}_FiberEndpoint{smths}.func.gii")
map_rh = op.join(regions_dir, f"rh.right{tract}_box_1mm_{regions}_FiberEndpoint{smths}.func.gii")
map_arr_lh = nib.load(map_lh).agg_data()
map_arr_rh = nib.load(map_rh).agg_data()

    # Remove medial wall
map_arr = _rm_medial_wall(
        map_arr_lh,
        map_arr_rh,
        space=SPACE,
        density=DENSITY,
        neuromaps_dir=neuromaps_dir,
)

print("main code:" , map_arr.shape)
    # Decode map
corrs_df = decode.transform([map_arr], method="correlation") 
#  DSECRIBE WHAT THIS DOES. I THINK IT DOES THIS. DO YOU THINK CODE AND DESCRIPTION MATCH? DO YOU THINK THIS IS APPROPRIATE ANALYSIS?
# COSINE DISTANCE? 
# VERTEX CORRELATION? WHAT IS VERTEX ON BRAIN?


filtered_df, filtered_features, filtered_frequencies = _decoding_filter(
    corrs_df,
    features,
    classification,
    freq_by_topic=frequencies,
    class_by_topic=class_lst,
)
filtered_df.columns = ["r"]
separated_results[f"{tract}_{regions}{smths}_thr-{thresholds}"] = filtered_df.sort_values(by="r", ascending=False)

# Visualize maps to decode
# Visualize results
corrs = filtered_df["r"].to_numpy()
features = filtered_features

# Sort features and correlations
corrs = np.array(corrs)
sorted_indices = np.argsort(-corrs)
corrs = corrs[sorted_indices]
features = np.array(features, dtype=object)[sorted_indices]

n_rows = min(len(corrs), 10)
corrs = corrs[:n_rows]
features = features[:n_rows]

main code: (299881,)


  0%|          | 0/1 [00:00<?, ?it/s]


In [82]:
print(map_arr.shape)
print(map_arr_rh.shape)
print(map_arr_lh.shape)

N_VERTICES_PH = 32492
map_arr_lh.shape[0] == N_VERTICES_PH

(299881,)
(163842,)
(163842,)


False

### Run decoder on each regions separate

In [55]:
sep_figures_dir = op.join(figures_dir, "separated")
os.makedirs(sep_figures_dir, exist_ok=True)

separated_results = {}
for fig_i, (threshold, tract, region, smth) in enumerate(itertools.product(thresholds, tracts, regions, smths)):
    # Path to the maps
    regions_dir = op.join(
        data_dir, 
        "white-matter-atlas_thresholds", 
        f"cortexmap_binarize_smooth-surf-1_threshold-{threshold}_dilate-0", 
        "cortexmap", 
        "func",
    )
    
    # Read maps
    map_lh = op.join(regions_dir, f"lh.left{tract}_box_1mm_{region}_FiberEndpoint{smth}.func.gii")
    map_rh = op.join(regions_dir, f"rh.right{tract}_box_1mm_{region}_FiberEndpoint{smth}.func.gii")
    map_arr_lh = nib.load(map_lh).agg_data()
    map_arr_rh = nib.load(map_rh).agg_data()

    # Remove medial wall
    map_arr = _rm_medial_wall(
        map_arr_lh,
        map_arr_rh,
        space=SPACE,
        density=DENSITY,
        neuromaps_dir=neuromaps_dir,
    )

    # Decode map
    corrs_df = decode.transform([map_arr], method="correlation")
    filtered_df, filtered_features, filtered_frequencies = _decoding_filter(
        corrs_df,
        features,
        classification,
        freq_by_topic=frequencies,
        class_by_topic=class_lst,
    )
    filtered_df.columns = ["r"]
    separated_results[f"{tract}_{region}{smth}_thr-{threshold}"] = filtered_df.sort_values(by="r", ascending=False)
    np.savetxt("CST_features.csv", filtered_features, fmt="%s", delimiter = ",")

    # Visualize maps to decode
    plot_surf_maps(
        map_arr_lh, 
        map_arr_rh, 
        space=SPACE, 
        density=DENSITY, 
        cmap="YlOrRd",
        color_range=(0, 1),
        title=f"{TRACTS_DICT[tract]} {REGIONS_DICT[region]}\n{SMTHS_DICT[smth]}. Threshold: {threshold}",
        data_dir=data_dir,
        out_fig=op.join(sep_figures_dir, f"{fig_i}-01_{tract}_{region}{smth}_thr-{threshold}_surf.png"),
    )

    # Visualize results
    corrs = filtered_df["r"].to_numpy()

    if not np.any(np.isnan(corrs)) and corrs.size > 0: # Skip one of the regions of CST
        # Radar plot
        plot_radar(
            corrs, 
            filtered_features, 
            MODEL,
            out_fig=op.join(sep_figures_dir, f"{fig_i}-02_{tract}_{region}{smth}_thr-{threshold}_radar.png"),
        )
        
        # Word cloud plot
        plot_cloud(
            corrs, 
            filtered_features,
            MODEL,
            frequencies=filtered_frequencies,
            out_fig=op.join(sep_figures_dir, f"{fig_i}-03_{tract}_{region}{smth}_thr-{threshold}_wordcloud.png"),
        )

  0%|          | 0/1 [00:00<?, ?it/s]
/home/sohmee/.local/share/virtualenvs/test-uar988OR/lib/python3.10/site-packages/numpy/lib/npyio.py:1393: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.asarray(X)
  0%|          | 0/1 [00:00<?, ?it/s]
/home/sohmee/.local/share/virtualenvs/test-uar988OR/lib/python3.10/site-packages/numpy/lib/npyio.py:1393: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.asarray(X)
/home/sohmee/.local/share/virtualenvs/test-uar988OR/lib/python3.10/site-packages/nimare/stats.py:109: RuntimeWarning: invalid value encounte

In [34]:
with open('CST_features.csv', 'w') as f:  # You will need 'wb' mode in Python 2.x
    w = csv.DictWriter(f, separated_results.keys())
    w.writeheader()
    w.writerow(separated_results)

In [10]:
sep_figures_dir = op.join(figures_dir, "separated")
os.makedirs(sep_figures_dir, exist_ok=True)

separated_results = {}
for tract in tracts:
    for fig_i, (threshold, region, smth) in enumerate(itertools.product(thresholds, regions, smths)):
        # Path to the maps
        regions_dir = op.join(
            data_dir, 
            "white-matter-atlas_thresholds", 
            f"cortexmap_binarize_smooth-surf-1_threshold-{threshold}_dilate-0", 
            "cortexmap", 
            "func",
        )
        
        # Read maps
        map_lh = op.join(regions_dir, f"lh.left{tract}_box_1mm_{region}_FiberEndpoint{smth}.func.gii")
        map_rh = op.join(regions_dir, f"rh.right{tract}_box_1mm_{region}_FiberEndpoint{smth}.func.gii")
        map_arr_lh = nib.load(map_lh).agg_data()
        map_arr_rh = nib.load(map_rh).agg_data()

        # Remove medial wall
        map_arr = _rm_medial_wall(
            map_arr_lh,
            map_arr_rh,
            space=SPACE,
            density=DENSITY,
            neuromaps_dir=neuromaps_dir,
        )

        # Decode map
        corrs_df = decode.transform([map_arr], method="correlation")
        filtered_df, filtered_features, filtered_frequencies = _decoding_filter(
            corrs_df,
            features,
            classification,
            freq_by_topic=frequencies,
            class_by_topic=class_lst,
        )
        filtered_df.columns = ["r"]
        separated_results[f"{tract}_{region}{smth}_thr-{threshold}"] = filtered_df.sort_values(by="r", ascending=False)

        # Visualize maps to decode
        plot_surf_maps(
            map_arr_lh, 
            map_arr_rh, 
            space=SPACE, 
            density=DENSITY, 
            cmap="YlOrRd",
            color_range=(0, 1),
            title=f"{TRACTS_DICT[tract]} {REGIONS_DICT[region]}\n{SMTHS_DICT[smth]}. Threshold: {threshold}",
            data_dir=data_dir,
            out_fig=op.join(sep_figures_dir, f"{fig_i}-01_{tract}_{region}{smth}_thr-{threshold}_surf.png"),
        )

        # Visualize results
        corrs = filtered_df["r"].to_numpy()
        if not np.any(np.isnan(corrs)) and corrs.size > 0: # Skip one of the regions of CST
            # Radar plot
            plot_radar(
                corrs, 
                filtered_features, 
                MODEL,
                out_fig=op.join(sep_figures_dir, f"{fig_i}-02_{tract}_{region}{smth}_thr-{threshold}_radar.png"),
            )
            
            # Word cloud plot
            plot_cloud(
                corrs, 
                filtered_features,
                MODEL,
                frequencies=filtered_frequencies,
                out_fig=op.join(sep_figures_dir, f"{fig_i}-03_{tract}_{region}{smth}_thr-{threshold}_wordcloud.png"),
            )

        del map_lh, map_rh, map_arr_lh, map_arr_rh, corrs_df, filtered_df, filtered_features, filtered_frequencies, corrs
gc.collect()

662

### Run decoder on combined regions for each tract

In [20]:
com_figures_dir = op.join(figures_dir, "combined")
os.makedirs(com_figures_dir, exist_ok=True)

combined_results = {}
for fig_i, (threshold, tract, smth) in enumerate(itertools.product(thresholds, tracts, smths)):
    # Path to the maps
    regions_dir = op.join(
        data_dir, 
        "white-matter-atlas_thresholds", 
        f"cortexmap_binarize_smooth-surf-1_threshold-{threshold}_dilate-0", 
        "cortexmap", 
        "func",
    )
    
    # Read maps
    map_lpi_lh = op.join(regions_dir, f"lh.left{tract}_box_1mm_LPI_FiberEndpoint{smth}.func.gii")
    map_lpi_rh = op.join(regions_dir, f"rh.right{tract}_box_1mm_LPI_FiberEndpoint{smth}.func.gii")
    map_ras_lh = op.join(regions_dir, f"lh.left{tract}_box_1mm_RAS_FiberEndpoint{smth}.func.gii")
    map_ras_rh = op.join(regions_dir, f"rh.right{tract}_box_1mm_RAS_FiberEndpoint{smth}.func.gii")
    
    map_lpi_arr_lh = nib.load(map_lpi_lh).agg_data()
    map_lpi_arr_rh = nib.load(map_lpi_rh).agg_data()
    map_ras_arr_lh = nib.load(map_ras_lh).agg_data()
    map_ras_arr_rh = nib.load(map_ras_rh).agg_data()

    # Combined regions for each tract
    map_arr_lh = np.maximum(map_lpi_arr_lh, map_ras_arr_lh) # Take the maximum to address overlap
    map_arr_rh = np.maximum(map_lpi_arr_rh, map_ras_arr_rh) # Take the maximum to address overlap
    
    # Remove medial wall
    map_arr = _rm_medial_wall(
        map_arr_lh,
        map_arr_rh,
        space=SPACE,
        density=DENSITY,
        neuromaps_dir=neuromaps_dir,
    )

    # Decode map
    corrs_df = decode.transform([map_arr], method="correlation")
    filtered_df, filtered_features, filtered_frequencies = _decoding_filter(
        corrs_df,
        features,
        classification,
        freq_by_topic=frequencies,
        class_by_topic=class_lst,
    )

    filtered_df.columns = ["r"]
    combined_results[f"{tract}{smth}_thr-{threshold}"] = filtered_df.sort_values(by="r", ascending=False)

    # Visualize maps to decode
    surf_fig = plot_surf_maps(
        map_arr_lh, 
        map_arr_rh, 
        space=SPACE, 
        density=DENSITY, 
        cmap="YlOrRd",
        color_range=(0, 1),
        title=f"{TRACTS_DICT[tract]} LPI+RAS\n{SMTHS_DICT[smth]}. Threshold: {threshold}",
        data_dir=data_dir,
        out_fig=op.join(com_figures_dir, f"{fig_i}-01_{tract}_LPI+RAS{smth}_thr-{threshold}_surf.png"),
    )

    # Visualize results
    corrs = filtered_df["r"].to_numpy()
    if not np.any(np.isnan(corrs)) and corrs.size > 0: # Skip one of the regions of CST
        # Radar plot
        plot_radar(
            corrs, 
            filtered_features, 
            MODEL,
            out_fig=op.join(com_figures_dir, f"{fig_i}-02_{tract}_LPI+RAS{smth}_thr-{threshold}_radar.png"),
        )

        # Word cloud plot
        plot_cloud(
            corrs, 
            filtered_features,
            MODEL,
            frequencies=filtered_frequencies,
            out_fig=op.join(com_figures_dir, f"{fig_i}-03_{tract}_LPI+RAS{smth}_thr-{threshold}_wordcloud.png"),
        )
    
gc.collect()

  0%|          | 0/1 [00:00<?, ?it/s]
/home/sohmee/.local/share/virtualenvs/test-uar988OR/lib/python3.10/site-packages/nimare/stats.py:109: RuntimeWarning: invalid value encountered in true_divide
  rs = temp / (datass[1:] * datass[0])
  0%|          | 0/1 [00:00<?, ?it/s]


115119

### Make figures

In [24]:
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.image as mpimg

In [25]:
comb_width, comb_hight = 25, 13
sep_width, sep_hight = 25, 25

n_comb_rows, n_comb_cols = 3, 6
n_sep_rows, n_sep_cols = 6, 6

In [26]:
fig_i = 0
for thr_i, threshold in enumerate(thresholds):
    fig = plt.figure(figsize=(sep_width, sep_hight))
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.1, hspace=0.1)
    gs = GridSpec(nrows=n_sep_rows, ncols=n_sep_cols, figure=fig)

    for trc_i, tract in enumerate(tracts):
        for reg_i, region in enumerate(regions):
            for smth_i, smth in enumerate(smths):
                surf_plt = op.join(sep_figures_dir, f"{fig_i}-01_{tract}_{region}{smth}_thr-{threshold}_surf.png")
                radar_plt = op.join(sep_figures_dir, f"{fig_i}-02_{tract}_{region}{smth}_thr-{threshold}_radar.png")
                wordcloud_plt = op.join(sep_figures_dir, f"{fig_i}-03_{tract}_{region}{smth}_thr-{threshold}_wordcloud.png")

                for img_i, img_file in enumerate([surf_plt, radar_plt, wordcloud_plt]):
                    ax = fig.add_subplot(gs[trc_i*2 + reg_i, smth_i*3 + img_i], aspect="equal")
                    if op.exists(img_file):
                        img = mpimg.imread(img_file)    
                        ax.imshow(img)

                    ax.set_axis_off()

                fig_i += 1

        out_file = op.join(figures_dir, f"results-separated_thr-{float(threshold):.2f}.png")
        fig.savefig(out_file, bbox_inches="tight", dpi=300)
        plt.close()

IndexError: index 6 is out of bounds for axis 0 with size 6

In [10]:
fig_i = 0
for thr_i, threshold in enumerate(thresholds):
    fig = plt.figure(figsize=(comb_width, comb_hight))
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.1, hspace=0.1)
    gs = GridSpec(nrows=n_comb_rows, ncols=n_comb_cols, figure=fig)

    for trc_i, tract in enumerate(tracts):
        for smth_i, smth in enumerate(smths):

            surf_plt = op.join(com_figures_dir, f"{fig_i}-01_{tract}_LPI+RAS{smth}_thr-{threshold}_surf.png")
            radar_plt = op.join(com_figures_dir, f"{fig_i}-02_{tract}_LPI+RAS{smth}_thr-{threshold}_radar.png")
            wordcloud_plt = op.join(com_figures_dir, f"{fig_i}-03_{tract}_LPI+RAS{smth}_thr-{threshold}_wordcloud.png")
            
            for img_i, img_file in enumerate([surf_plt, radar_plt, wordcloud_plt]):
                ax = fig.add_subplot(gs[trc_i, smth_i*3 + img_i], aspect="equal")
                if op.exists(img_file):
                    img = mpimg.imread(img_file)    
                    ax.imshow(img)
                
                ax.set_axis_off()

            fig_i += 1

    out_file = op.join(figures_dir, f"results-combined_thr-{float(threshold):.2f}.png")
    fig.savefig(out_file, bbox_inches="tight", dpi=300)
    plt.close()